In [8]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.stats as stats
import matplotlib.pyplot as plt
import umap
import pickle
import time

from sklearn import linear_model, model_selection, metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as GDA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV,train_test_split,cross_val_score
from sklearn.metrics import roc_auc_score as RAS
from sklearn.metrics import roc_curve as ROC
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier as KNC

`jupyter nbconvert --to latex "Race and Incarceration in America.ipynb" --TagRemovePreprocessor.remove_input_tags='{"hide_input"}'; pdflatex "Race and Incarceration in America"`

# Introduction

Last semester I took a look a data set containing the information of more than 7.5 million individuals that have been processed by the criminal justice system. I found that racial minorities were more likely to receive extreme sentences, agreeing with existing research around bias in the criminal justice system. In this project I will be exploring the data from a machine learning perspective. My goal is to determine if this data can be classified in such a way that is predictive of race.

# Possible questions
can we use ml techniques to correctly classify this data?
which ones fail and why?
can we create a predictive model for sentence lengths?
should sentencing be offloaded to a ml algorithm.

TSNE is having mized results while UMAP doesn't seem to capture any good groupings. Maybe I should go straight to kmeans or kdtrees.
I should block by crime and examine a few "big" crimes the try several different perplexities.

In [9]:
chunksize = 100000
rdf = pd.read_csv('regression_df.csv', chunksize=chunksize)

In [10]:
cols = ['RACE','GENDER','AGE','OFFENSE','FACILITY','DETAINER','SENTENCE DAYS']
df = pd.read_csv('individuals.csv',usecols=cols)

In [11]:
samp = df.sample(20000)
samp.RACE = pd.factorize(samp['RACE'])[0] + 1
samp.GENDER = pd.factorize(samp['GENDER'])[0] + 1
samp.OFFENSE = pd.factorize(samp['OFFENSE'])[0] + 1
samp.DETAINER = pd.factorize(samp['DETAINER'])[0] + 1
samp.FACILITY = pd.factorize(samp['FACILITY'])[0] + 1
samp_y = samp.RACE
samp_X = samp[['GENDER','AGE','OFFENSE','FACILITY','DETAINER','SENTENCE DAYS']]

In [12]:
param_grid = {
    'n_estimators': np.arange(100,400,20),
    'max_depth': np.arange(10,100,10),
    'max_features': np.arange(1,6)
}
def oob(clf, X, y):
    return clf.oob_score_

clf = RandomForestClassifier(oob_score=True)

s = time.time()
clf = GridSearchCV(clf, param_grid, scoring=oob, cv=5)
clf = clf.fit(samp_X, samp_y)
e = time.time()
print(f'time to train is {(e-s)/60} minutes')

clf = clf.best_estimator_
clf = clf.fit(samp_X, samp_y)
print(f'oob score is {clf.oob_score_}')

with open('RandonForestClf.pickle', "wb+") as f:
    pickle.dump(clf, f)

KeyboardInterrupt: 

In [7]:
with open('RandonForestClf.pickle',"rb") as f:
    clf = pickle.load(f)
print(clf.oob_score_)
print(clf.feature_importances_)

EOFError: Ran out of input

In [11]:
samp = df.sample(200000)
samp.RACE = pd.factorize(samp['RACE'])[0] + 1
samp.GENDER = pd.factorize(samp['GENDER'])[0] + 1
samp.OFFENSE = pd.factorize(samp['OFFENSE'])[0] + 1
samp.DETAINER = pd.factorize(samp['DETAINER'])[0] + 1
samp.FACILITY = pd.factorize(samp['FACILITY'])[0] + 1
samp_y = samp.RACE
samp_X = samp[['GENDER','AGE','OFFENSE','FACILITY','DETAINER','SENTENCE DAYS']]

In [12]:
clf.score(samp_X, samp_y)

0.340885

In [13]:
samp = df.sample(2000)
samp.RACE = pd.factorize(samp['RACE'])[0] + 1
samp.GENDER = pd.factorize(samp['GENDER'])[0] + 1
samp.OFFENSE = pd.factorize(samp['OFFENSE'])[0] + 1
samp.DETAINER = pd.factorize(samp['DETAINER'])[0] + 1
samp.FACILITY = pd.factorize(samp['FACILITY'])[0] + 1
samp_y = samp.RACE
samp_X = samp[['GENDER','AGE','OFFENSE','FACILITY','DETAINER','SENTENCE DAYS']]